# <u>**Netflix Subscription Type Prediction**</u>

## Step 1: Data Loading and Pre-processing

First, we need to install the required libraries and import them.

In [3]:
# Installing necessary libraries
!pip install pandas
!pip install numpy
!pip install scikit-learn


# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 53.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 49.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


Next, we load the dataset and take a look at the first few rows of the dataset.

In [5]:
# Load the dataset
df = pd.read_csv('/Users/rwilcox/Projects/Netflix Project/Netflix Userbase.csv')

# Take a look at the first few rows of the DataFrame.
print(df.head())


   User ID Subscription Type  Monthly Revenue Join Date Last Payment Date  \
0        1             Basic               10  15-01-22          10-06-23   
1        2           Premium               15  05-09-21          22-06-23   
2        3          Standard               12  28-02-23          27-06-23   
3        4          Standard               12  10-07-22          26-06-23   
4        5             Basic               10  01-05-23          28-06-23   

          Country  Age  Gender      Device Plan Duration  
0   United States   28    Male  Smartphone       1 Month  
1          Canada   35  Female      Tablet       1 Month  
2  United Kingdom   42    Male    Smart TV       1 Month  
3       Australia   51  Female      Laptop       1 Month  
4         Germany   33    Male  Smartphone       1 Month  


Now, we'll perform some basic pre-processing.  Let's check if there are any missing values in the data. If there are, we'll need to make a decision on how to handle them - for example, we could remove those rows or columns, or fill the missing values with a specified method (like the mean or the median of that column).

In [6]:
# Check for missing values
print(df.isnull().sum())

User ID              0
Subscription Type    0
Monthly Revenue      0
Join Date            0
Last Payment Date    0
Country              0
Age                  0
Gender               0
Device               0
Plan Duration        0
dtype: int64


Since there are no missing values, next we need to convert categorical data to numerical, as machine learning algorithms work better with numerical data. We'll use LabelEncoder from sklearn.preprocessing to do this.